# Neural network based regression - Fully connected network with clustering

In [ ]:
import os
import sys

sys.path.append(os.path.join(os.path.abspath(""), ".."))

from torch import nn, optim

from nnbma.networks import FullyConnected, PolynomialNetwork, MergingNetwork
from nnbma.layers import PolynomialExpansion
from nnbma.learning import LearningParameters, MaskedMSELoss

from helpers.preprocessing import get_names, prepare_clusters
from helpers.training import procedure
from helpers.results import save_results

In [ ]:
filename = os.path.join(
    os.path.splitext(os.path.abspath(""))[0], "out-nn-regression-fc"
)

### Selected inputs (can be modified)

In [ ]:
lines = None  # If None, select all lines by default

In [ ]:
inputs_names, outputs_names = get_names(lines=lines)
n_inputs, n_outputs = len(inputs_names), len(outputs_names)

### Clusters

`n_comps` is the number of principal components from `clustering.ipynb`.

In [ ]:
n_clusters = 4
n_comps = [500, 100, 75, 350]

### Architecture settings (can be modified)

In [ ]:
n_hidden_layers = 3
last_layer_size = 1000
other_layers_size = 1000
poly_degree = 3

In [ ]:
## Architecture hyperparameters

activation = nn.ELU()
batch_norm = False

subnetworks = []
w = n_outputs * [1.0]

for k in range(n_clusters):
    _lines = prepare_clusters(n_clusters)[k + 1]
    n_comp = n_comps[k]

    _, _outputs_names = get_names(lines=_lines)
    _n_outputs = len(_outputs_names)

    n_expanded_inputs = PolynomialExpansion.expanded_features(poly_degree, n_inputs)

    _layers_sizes = [n_expanded_inputs] + n_hidden_layers * [n_comp] + [_n_outputs]

    subnetworks.append(
        PolynomialNetwork(
            n_inputs,
            poly_degree,
            FullyConnected(
                _layers_sizes,
                activation,
                batch_norm=batch_norm,
                outputs_names=_outputs_names,
            ),
            inputs_names=inputs_names,
            outputs_names=_outputs_names,
        )
    )

    for i in range(n_outputs):
        line = outputs_names[i]
        if line in _outputs_names:
            w[i] = n_outputs / _n_outputs

use_mask = True

## Network creation

model = MergingNetwork(
    subnetworks,
    inputs_names=inputs_names,
    outputs_names=outputs_names,
    inputs_transformer=None,  # Will be set in the procedure
    outputs_transformer=None,  # Will be set in the procedure
)

### Training settings (can be modified)

In [ ]:
# Epochs
epochs = 200

# Batch size
batch_size = 500

# Loss function
use_mask = True
loss = MaskedMSELoss() if use_mask else nn.MSELoss()

# Optimizer
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), learning_rate)

# Scheduler
factor = 0.9
patience = 5
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=patience, factor=factor, min_lr=learning_rate * 1e-3
)

In [ ]:
learning_params = LearningParameters(loss, epochs, batch_size, optimizer, scheduler)

### Training procedure

In [ ]:
results = procedure(
    outputs_names,
    model,
    learning_params,
    use_mask,
    verbose=True,
)

### Saving results

In [ ]:
plot_profile = True

In [ ]:
arch_name = f"hidd_{n_hidden_layers}_last_{last_layer_size}_other_{other_layers_size}_deg_{poly_degree}"

save_results(
    results,
    outputs_names,
    model,
    learning_params,
    use_mask,
    filename,
    architecture_name=arch_name,
    plot_profiles=plot_profile,
)